# Calibre

> Calibre library management for Ragfood

In [ ]:
#| default_exp calibre

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, subprocess, json, tempfile, re
from ipywidgets import HTML
from Ragfood.mediator import Colleague
from Ragfood.states import *

In [ ]:
#| export
class Calibre(Colleague):
    """The class represents a Calibre library and provides functions for handling it (book list, export).
    
    You can specify export and library folders. After execution, you have access to the library (uses the calibredb command).
    For testing purposes, you can create a sample library with four free books—simply enable the switch.
    Creates a custom column in calibre: Embeddings to store informations.
    """
    
    # Static attributes
    export_pdf_to_folder = '~/labor/ragfood_pdf'
    calibre_library_folder = '~/labor/test_calibre_with_4_pdfs'
    create_sample_library_with_4_books_if_library_folder_not_exists = True
    
    @staticmethod
    def create_library_with_4_books(library_folder=None):
        """Create a sample library with 4 free books for testing."""
        if library_folder is None:
            library_folder = Calibre.calibre_library_folder
        
        # temp folder
        tf = tempfile.gettempdir()
        
        # create calibre library with 4 books if library folder is empty and switch is on
        if Calibre.create_sample_library_with_4_books_if_library_folder_not_exists and not os.path.exists(os.path.expanduser(Calibre.calibre_library_folder)):
            # get some free pdfs from github.com/nivu/ai_all_resources and arxiv.org
            os.system(f"""[ -f {tf}/mathematics_for_machine_learning.pdf ] || curl -o {tf}/mathematics_for_machine_learning.pdf https://mml-book.github.io/book/mml-book.pdf""")
            os.system(f"""[ -f {tf}/getting_started_with_artificial_intelligence.pdf ] || curl -o {tf}/getting_started_with_artificial_intelligence.pdf https://www.ibm.com/downloads/documents/us-en/107a02e94bc8f761""")
            os.system(f"""[ -f {tf}/attention_is_all_you_need.pdf ] || curl -o {tf}/attention_is_all_you_need.pdf https://arxiv.org/pdf/1706.03762""")
            os.system(f"""[ -f {tf}/an_introduction_to_convolutional_neural_networks.pdf ] || curl -o {tf}/an_introduction_to_convolutional_neural_networks.pdf https://arxiv.org/pdf/1511.08458""")
            
            # create empty library and add books
            os.system(f"""mkdir {library_folder} && mkdir {library_folder}/.calnotes && mkdir {library_folder}/.caltrash""")
            os.system(f"""calibredb restore_database --really-do-it --library-path={library_folder} 1>/dev/null 2>&1""")
            os.system(f"""calibredb add -t 'MATHEMATICS FOR MACHINE LEARNING' {tf}/mathematics_for_machine_learning.pdf --library-path={library_folder} 1>/dev/null 2>&1""")
            os.system(f"""calibredb add -t 'Getting Started with Artificial Intelligence' {tf}/getting_started_with_artificial_intelligence.pdf --library-path={library_folder} 1>/dev/null 2>&1""")
            os.system(f"""calibredb add -t 'Attention Is All You Need' {tf}/attention_is_all_you_need.pdf -a "Ashish Vaswani & Noam Shazeer & Niki Parmar & Jakob Uszkoreit & Llion Jones & Aidan N. Gomez & Łukasz Kaiser" --library-path={library_folder} 1>/dev/null 2>&1""")
            os.system(f"""calibredb add -t 'An Introduction to Convolutional Neural Networks' -a "Keiron O'Shea & Ryan Nash" {tf}/an_introduction_to_convolutional_neural_networks.pdf --library-path={library_folder} 1>/dev/null 2>&1""")
    
    @staticmethod
    def create_column_embedding(library_folder):
        """Create custom embedding column in Calibre library."""
        try:
            custom_columns_output = subprocess.run(["calibredb", "--library-path=" + library_folder, "custom_columns", "-d"], check=True, capture_output=True, text=True).stdout
            if "'label': 'embedding'" not in custom_columns_output:
                subprocess.run(["calibredb", "--library-path=" + library_folder, "add_custom_column", "embedding", "Embedding", "text"], check=True, capture_output=True)
        except:
            pass
    
    def __init__(self, title='Calibre', mediator=None, library_folder=None, export_folder=None):
        """Initialize Calibre instance."""
        # attributes and super constructor
        super().__init__(mediator)
        self.title = title
        if library_folder:
            Calibre.calibre_library_folder = library_folder
        if export_folder:
            Calibre.export_pdf_to_folder = export_folder
        
        # check calibredb exists
        try:
            subprocess.run(["calibredb", "--version"], check=True, capture_output=True, text=True)
            test_ok = True
        except (subprocess.CalledProcessError, FileNotFoundError):
            test_ok = False
        
        if test_ok:
            # check calibre_library exists
            test_ok = os.path.exists(os.path.expanduser(Calibre.calibre_library_folder+'/.calnotes'))
            if test_ok:
                # export folder
                os.makedirs(os.path.expanduser(Calibre.export_pdf_to_folder), exist_ok=True)
                # create custom column embedding if not exists
                Calibre.create_column_embedding(Calibre.calibre_library_folder)
            # no library
            else:
                if Calibre.create_sample_library_with_4_books_if_library_folder_not_exists:
                    Calibre.create_library_with_4_books()
                    Calibre.create_column_embedding(Calibre.calibre_library_folder)
                else:
                    # i don't like exceptions
                    self.changed('OnCalibreCreated', RAGFOOD_ERROR_LIBNOTFOUND)
            
            # tell mediator
            self.changed('OnCalibreCreated', RAGFOOD_OK)
        else:
            self.changed('OnCalibreCreated', RAGFOOD_ERROR_CALIBRENOTFOUND)
        
        # get books
        self.loadBooklist()
        
        # draw head line
        self.buildInfoView()
    
    def buildInfoView(self):
        """Build the info widget."""
        # widgets
        cyanbox = lambda t, c: "<td align=center>"+t+"</td><td align='center'><p style='background-color:#B0E0E6'>&nbsp;&nbsp;"+c+"&nbsp;&nbsp;</p></td>"
        hm = f"""<table width='100%'><tr><td ><font size=+2 color=#005F6A><b><p>{self.title}</b></font></td><td align=right><table><tr>"""
        hm += cyanbox('Books', str(len(self.book_list))) + cyanbox('Folder', Calibre.calibre_library_folder) + cyanbox('Export', Calibre.export_pdf_to_folder)
        hm += """</tr></table></td></tr></table>"""
        self.widget = HTML(hm, layout={'width': 'auto'})
    
    def loadBooklist(self):
        """Load the book list from Calibre library."""
        # get booklist
        asFileName = lambda s: re.sub(r'[^a-z0-9_.]+', '', re.sub(r'[\s]+', '_', s.lower()))
        listcmd_output = subprocess.run(["calibredb", "--library-path=" + Calibre.calibre_library_folder, "list", "--fields", "title,authors,formats,*embedding", "--for-machine"], check=True, capture_output=True, text=True).stdout
        self.book_list = json.loads(listcmd_output)
        
        # filter: only pdf format
        self.book_list = [book for book in self.book_list if len(['x' for f in book['formats'] if f.split('.')[-1]=='pdf']) > 0]
        
        # init embedding collumn and add filename
        for book in self.book_list:
            # filename
            book['filename'] = asFileName(book['title']) + '.pdf'
            # init embedding value
            if not '*embedding' in book:
                book['embedding'] = None
    
    def export(self, calibre_ids, export_folder=None):
        """Export books by Calibre IDs."""
        # create filename from title
        asFileName = lambda s: re.sub(r'[^a-z0-9_.]+', '', re.sub(r'[\s]+', '_', s.lower()))
        
        # new exportfolder if passed
        if not export_folder:
            export_folder = os.path.expanduser(Calibre.export_pdf_to_folder)
        else:
            Calibre.export_pdf_to_folder = os.path.expanduser(export_folder)
        
        # get valid ids
        valid_ids_output = subprocess.run(["calibredb", "list", "--library-path=" + Calibre.calibre_library_folder, "-f", "id"], check=True, capture_output=True, text=True).stdout.splitlines()
        valid_ids = valid_ids_output[1:]  # Skip header line
        
        # check if id list empty
        if not calibre_ids:
            self.changed('OnCalibreNoValidCalibreID', RAGFOOD_ERROR_CALIBRE_ID_UNKNOWN, calibre_ids, valid_ids=valid_ids)
        else:
            # for every calibre_id
            for calibre_id in calibre_ids:
                # check valid
                if not str(calibre_id) in valid_ids:
                    self.changed('OnCalibreNoValidCalibreID', RAGFOOD_ERROR_CALIBRE_ID_UNKNOWN, calibre_id, valid_ids=valid_ids)
                else:
                    # only if not exists
                    filename = self.getPathByID(calibre_id)
                    
                    if not os.path.exists(filename.replace('.pdf', '.opf')) or True:
                        # export pdf and rename
                        cmd = ["calibredb", "export", "--library-path=" + Calibre.calibre_library_folder, "--template={id} ",
                               "--dont-save-cover", "--to-dir=" + export_folder, str(calibre_id)]
                        subprocess.run(cmd, check=True)
                        subprocess.run(["mv", f"{export_folder}/{calibre_id}.pdf", f"{self.getPathByID(calibre_id)}"])
                        subprocess.run(["mv", f"{export_folder}/{calibre_id}.opf", f"{self.getPathByID(calibre_id).replace('.pdf', '.opf')}"])
                    
                    # export ok - trigger event even if files exists
                    self.changed('OnCalibreExport', RAGFOOD_OK, calibre_id,
                                library_path=Calibre.calibre_library_folder,
                                export_path=export_folder,
                                files=[self.getPathByID(calibre_id), self.getPathByID(calibre_id).replace('.pdf', '.opf')])
    
    def getPathByID(self, calibre_id):
        """Get file path by Calibre ID."""
        # get path
        for book in self.book_list:
            if book['id'] == calibre_id:
                return os.path.expanduser(Calibre.export_pdf_to_folder+'/'+book['filename'])
        return ''

## Example Usage

In [ ]:
# Example usage (requires Calibre to be installed)
# calibre = Calibre()
# display(calibre.widget)